<a href="https://colab.research.google.com/github/slava-ugolnikov/KG_practice_2025/blob/main/main_failed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nested NER: Classical and LLM-driven approaches of extraction

## Dataset

The GENIA corpus is the primary collection of biomedical literature compiled and annotated within the scope of the GENIA project. The corpus was created to support the development and evaluation of information extraction and text mining systems for the domain of molecular biology.

The corpus contains 1,999 Medline abstracts, selected using a PubMed query for the three MeSH terms “human”, “blood cells”, and “transcription factors”. The corpus has been annotated with various levels of linguistic and semantic information.

In [ ]:
%pip install opendatasets

In [ ]:
import opendatasets as od

od.download(
    "https://www.kaggle.com/datasets/nishanthsalian/genia-biomedical-event-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: slavaugolnikov
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/nishanthsalian/genia-biomedical-event-dataset


In [40]:
import pandas as pd

genia_train = pd.read_csv('/content/genia-biomedical-event-dataset/train_data.csv')
genia_train.head()

,Sentence,TriggerWord,TriggerWordLoc,EventType
0,Down-regulation of interferon regulatory fact...,Down-regulation;expression;,1;8;,Negative_regulation;Gene_expression;
1,Although the bcr - abl translocation has been...,deregulation;,30;,Regulation;
2,Promoter methylation of CpG target sites or d...,NaN,NaN,NaN
3,"Therefore , we investigated whether IRF-4 pro...",regulation;expression;,16;19;,Regulation;Gene_expression;
4,Whereas promoter mutations or structural rear...,altered;expression;influence;transcription;,14;16;30;32;,Regulation;Gene_expression;Regulation;Transcri...


In [ ]:
genia_train.Sentence[0]

' Down-regulation of interferon regulatory factor 4 gene expression in leukemic cells due to hypermethylation of CpG motifs in the promoter region'

In [45]:
genia_train.Sentence[4]

' Whereas promoter mutations or structural rearrangements could be excluded as a cause of altered IRF-4 expression in hematopoietic cells , the IRF-4 promoter methylation status was found to significantly influence IRF-4 transcription .'

In [42]:
genia_train.count()

,0
Sentence,8666
TriggerWord,3709
TriggerWordLoc,3709
EventType,3709


In [ ]:
genia_test = pd.read_csv('/content/genia-biomedical-event-dataset/test_data.csv')
genia_test.head()

,Sentence,TriggerWord,TriggerWordLoc,EventType
0,Resistance to IL-10 inhibition of interferon ...,NaN,NaN,NaN
1,IL-10 has been shown to block the antigen-spe...,NaN,NaN,NaN
2,We found that peripheral blood CD4 + T cells ...,NaN,NaN,NaN
3,The phosphorylation of signal transducer and ...,NaN,NaN,NaN
4,Sera from RA patients induced signal transduc...,NaN,NaN,NaN


In [44]:
genia_test.count()

,0
Sentence,3360
TriggerWord,0
TriggerWordLoc,0
EventType,0


In [38]:
genia_train.EventType.unique()[:10]

array(['Negative_regulation;Gene_expression;', 'Regulation;', nan,
       'Regulation;Gene_expression;',
       'Regulation;Gene_expression;Regulation;Transcription;',
       'Gene_expression;Positive_regulation;Positive_regulation;',
       'Gene_expression;Gene_expression;', 'Negative_regulation;',
       'Positive_regulation;Gene_expression;',
       'Gene_expression;Positive_regulation;'], dtype=object)

In [ ]:
len(genia_train.EventType.unique())

571

There are 571 unique named entities in the train  

## What is to be done

Cloning github repository and installing requirements

In [ ]:
!git clone  https://github.com/foxlf823/sodner
%cd sodner

Cloning into 'sodner'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 54 (delta 11), reused 54 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (54/54), 612.09 KiB | 1.59 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/sodner


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 6.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


Downloading the PyTorch AllenNLP version of SciBERT

In [ ]:
!wget https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_cased.tar

--2025-04-21 07:02:06--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_cased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 3.5.79.211, 52.218.181.0, 52.92.202.184, ...
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|3.5.79.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 410521600 (392M) [application/x-tar]
Saving to: ‘scibert_scivocab_cased.tar’

scibert_scivocab_ca 100%[===================>] 391.50M  34.9MB/s    in 11s     

2025-04-21 07:02:18 (35.1 MB/s) - ‘scibert_scivocab_cased.tar’ saved [410521600/410521600]



In [ ]:
!tar -xvf scibert_scivocab_cased.tar

scibert_scivocab_cased/
scibert_scivocab_cased/weights.tar.gz
scibert_scivocab_cased/vocab.txt


## Applying span-based method

## Visualisation

## Конвертация

In [27]:
import pandas as pd
import nltk
import json
from nltk import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')

df = pd.read_csv("/content/genia-biomedical-event-dataset/train_data.csv")

output = []

for idx, row in df.iterrows():
    sent = row["Sentence"]
    trigger_words = row["TriggerWord"]
    trigger_locs = row["TriggerWordLoc"]
    event_types = row["EventType"]
    trigger_words

    tokens = nltk.word_tokenize(sent)

    entities = []
    if isinstance(trigger_locs, str):
        trigger_locs_list = trigger_locs.split(';')
        for i in range(len(trigger_locs_list)):
            try:
                start_token_idx = int(trigger_locs[i])
                end_token_idx = start_token_idx + 1
                token_start_char = len(" ".join(tokens[:start_token_idx])) + (1 if start_token_idx > 0 else 0)
                token_end_char = len(" ".join(tokens[:end_token_idx]))
                label = event_types[i] if i < len(event_types) else "EVENT"

                entities.append({
                    "start": token_start_char,
                    "end": token_end_char,
                    "label": label
                })
            except:
                continue

    output.append({
        "text": sent,
        "entities": entities
    })

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [37]:
df.head()

,Sentence,TriggerWord,TriggerWordLoc,EventType
0,Down-regulation of interferon regulatory fact...,Down-regulation;expression;,1;8;,Negative_regulation;Gene_expression;
1,Although the bcr - abl translocation has been...,deregulation;,30;,Regulation;
2,Promoter methylation of CpG target sites or d...,NaN,NaN,NaN
3,"Therefore , we investigated whether IRF-4 pro...",regulation;expression;,16;19;,Regulation;Gene_expression;
4,Whereas promoter mutations or structural rear...,altered;expression;influence;transcription;,14;16;30;32;,Regulation;Gene_expression;Regulation;Transcri...


In [31]:
output[:5]

[{'text': ' Down-regulation of interferon regulatory factor 4 gene expression in leukemic cells due to hypermethylation of CpG motifs in the promoter region',
  'entities': [{'start': 16, 'end': 18, 'label': 'N'},
   {'start': 66, 'end': 68, 'label': 'g'}]},
 {'text': ' Although the bcr - abl translocation has been shown to be the causative genetic aberration in chronic myeloid leukemia ( CML ) , there is mounting evidence that the deregulation of other genes , such as the transcription factor interferon regulatory factor 4 ( IRF-4 ) , is also implicated in the pathogenesis of CML .',
  'entities': [{'start': 17, 'end': 18, 'label': 'R'},
   {'start': 0, 'end': 8, 'label': 'e'}]},
 {'text': ' Promoter methylation of CpG target sites or direct deletions/insertions of genes are mechanisms of a reversible or permanent silencing of gene expression , respectively .',
  'entities': []},
 {'text': ' Therefore , we investigated whether IRF-4 promoter methylation or mutation may be involved in 

In [ ]:


with open("data/train.json", "w") as f:
    json.dump(output, f, indent=2)
